**Ejercicio 02: Cálculo de la Matriz TF-IDF y Búsqueda de Consultas en un Corpus**

El objetivo de este ejercicio es calcular la matriz TF-IDF de un corpus de documentos y luego aplicar una serie de consultas para recuperar los documentos más relevantes. Este ejercicio te ayudará a comprender cómo funciona el modelo de espacio vectorial y cómo se utiliza TF-IDF para ponderar términos en documentos y consultas.

In [33]:
# Librerias Necesarias
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**1. Preprocesamiento del texto:**

Lectura del corpus desde el archivo TXT.

In [34]:
# Leer el archivo TXT con el corpus desde la carpeta Datasets
with open('/kaggle/input/02tfidfmatrix-corpus/02tfidfmatrix_corpus.txt', 'r', encoding='utf-8') as file:
    corpus = file.readlines()

# Imprimir el contenido del corpus
print("Contenido del corpus:")
for idx, doc in enumerate(corpus):
    print(f"Documento {idx + 1}: {doc.strip()}")

Contenido del corpus:
Documento 1: Documento 1: La inteligencia artificial continúa avanzando rápidamente, transformando sectores como la salud y las finanzas. Las empresas están adoptando algoritmos de aprendizaje automático para mejorar la eficiencia. Sin embargo, el desafío principal sigue siendo garantizar que las decisiones basadas en datos sean justas y no perpetúen sesgos. La ética es fundamental en este contexto.
Documento 2: Documento 2: El desarrollo de videojuegos ha alcanzado un nuevo nivel con tecnologías como la realidad aumentada y la inteligencia artificial. Los jugadores ahora pueden interactuar en mundos virtuales más inmersivos. Este crecimiento también impulsa el mercado de los e-sports, donde las competencias profesionales atraen a millones de espectadores en todo el mundo.
Documento 3: Documento 3: El comercio electrónico ha cambiado la forma en que compramos. Con la creciente demanda de compras en línea, las empresas están optimizando sus plataformas digitales pa

**Tokenización de los documentos.**

In [35]:
def tokenizar(text):
    # Usar expresiones regulares para dividir el texto en palabras
    tokens = re.findall(r'\b\w+\b', text.lower())  # Convertimos a minúsculas y extraemos palabras
    return tokens

# Aplicar la tokenización a cada documento en el corpus
corpus_tokenizado = [tokenizar(doc) for doc in corpus]

# Imprimir el corpus tokenizado
print("Corpus tokenizado:")
for idx, tokens in enumerate(corpus_tokenizado):
    print(f"Documento {idx + 1}: {tokens}")

Corpus tokenizado:
Documento 1: ['documento', '1', 'la', 'inteligencia', 'artificial', 'continúa', 'avanzando', 'rápidamente', 'transformando', 'sectores', 'como', 'la', 'salud', 'y', 'las', 'finanzas', 'las', 'empresas', 'están', 'adoptando', 'algoritmos', 'de', 'aprendizaje', 'automático', 'para', 'mejorar', 'la', 'eficiencia', 'sin', 'embargo', 'el', 'desafío', 'principal', 'sigue', 'siendo', 'garantizar', 'que', 'las', 'decisiones', 'basadas', 'en', 'datos', 'sean', 'justas', 'y', 'no', 'perpetúen', 'sesgos', 'la', 'ética', 'es', 'fundamental', 'en', 'este', 'contexto']
Documento 2: ['documento', '2', 'el', 'desarrollo', 'de', 'videojuegos', 'ha', 'alcanzado', 'un', 'nuevo', 'nivel', 'con', 'tecnologías', 'como', 'la', 'realidad', 'aumentada', 'y', 'la', 'inteligencia', 'artificial', 'los', 'jugadores', 'ahora', 'pueden', 'interactuar', 'en', 'mundos', 'virtuales', 'más', 'inmersivos', 'este', 'crecimiento', 'también', 'impulsa', 'el', 'mercado', 'de', 'los', 'e', 'sports', 'donde'

**Normalización del texto (conversión a minúsculas, eliminación de signos de puntuación).**

In [36]:
def normalizar(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar signos de puntuación utilizando expresiones regulares
    text = re.sub(r'[^\w\s]', '', text)
    
    # Eliminar tildes
    # Crear una tabla de traducción para eliminar tildes
    tabla = str.maketrans(
        "áéíóúüñ",  # caracteres originales
        "aeiouun"   # caracteres reemplazados
    )
    text = text.translate(tabla)
    
    return text

# Aplicar la normalización a cada documento en el corpus
corpus_normalizado = [normalizar(doc) for doc in corpus]

# Imprimir el corpus normalizado
print("Corpus normalizado:")
for idx, doc in enumerate(corpus_normalizado):
    print(f"Documento {idx + 1}: {doc}")

Corpus normalizado:
Documento 1: documento 1 la inteligencia artificial continua avanzando rapidamente transformando sectores como la salud y las finanzas las empresas estan adoptando algoritmos de aprendizaje automatico para mejorar la eficiencia sin embargo el desafio principal sigue siendo garantizar que las decisiones basadas en datos sean justas y no perpetuen sesgos la etica es fundamental en este contexto

Documento 2: documento 2 el desarrollo de videojuegos ha alcanzado un nuevo nivel con tecnologias como la realidad aumentada y la inteligencia artificial los jugadores ahora pueden interactuar en mundos virtuales mas inmersivos este crecimiento tambien impulsa el mercado de los esports donde las competencias profesionales atraen a millones de espectadores en todo el mundo

Documento 3: documento 3 el comercio electronico ha cambiado la forma en que compramos con la creciente demanda de compras en linea las empresas estan optimizando sus plataformas digitales para ofrecer mejor

**2. Construcción de la matriz TF-IDF**

Cálculo de la frecuencia de término (TF) para cada término en cada documento.

In [37]:
# Crear el vectorizador para calcular la frecuencia de términos
vectorizer = CountVectorizer()

# Aplicar el vectorizador al corpus normalizado para obtener la matriz TF
tf_matrix = vectorizer.fit_transform(corpus_normalizado)

# Convertir la matriz a un DataFrame para facilitar la visualización
tf_df = pd.DataFrame(tf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Imprimir la matriz TF
print("Matriz TF (Frecuencia de Términos):")
print(tf_df)

Matriz TF (Frecuencia de Términos):
   acceder  accesibles  acceso  ademas  adopcion  adoptando  ahora  alcanzado  \
0        0           0       0       0         0          1      0          0   
1        0           0       0       0         0          0      1          1   
2        0           0       0       0         0          0      0          0   
3        1           0       0       0         0          0      0          0   
4        0           0       2       1         0          0      0          0   
5        0           0       0       0         0          0      0          0   
6        0           0       0       0         1          0      0          0   
7        0           1       0       0         0          0      0          0   

   algoritmos  anos  ...  una  uno  usuario  utilizan  vez  viaje  vida  \
0           1     0  ...    0    0        0         0    0      0     0   
1           0     0  ...    0    0        0         0    0      0     0   
2        

**Visualización de terminos en orden de frecuencia, simplemente para comprobar que los terminos que más se repiten dentro del corpus serian las stopword.**

In [38]:
# Sumar las frecuencias de cada término en todos los documentos
term_frecuencia = tf_df.sum().sort_values(ascending=False)

# Convertir a un DataFrame para una mejor visualización
term_frecuencia_df = pd.DataFrame(term_frecuencia, columns=['Frecuencia']).reset_index()
term_frecuencia_df.columns = ['Término', 'Frecuencia']

# Imprimir los términos ordenados por frecuencia
print("Términos ordenados por frecuencia:")
print(term_frecuencia_df)

Términos ordenados por frecuencia:
          Término  Frecuencia
0              la          25
1              de          20
2              el          18
3              en          15
4             los          11
..            ...         ...
223         etica           1
224  evolucionado           1
225   experiencia           1
226  experiencias           1
227      visuales           1

[228 rows x 2 columns]


**Cálculo de la frecuencia inversa de documento (IDF) para cada término en el corpus.**

In [39]:
# Número total de documentos en el corpus
N = len(corpus_normalizado)

# Calcular el número de documentos en los que aparece cada término
doc_frecuencia = (tf_df > 0).sum()

# Calcular el IDF usando la fórmula
idf_values = np.log(N / doc_frecuencia)

# Convertir a un DataFrame para facilitar la visualización
idf_df = pd.DataFrame(idf_values, columns=['IDF']).reset_index()
idf_df.columns = ['Término', 'IDF']

# Imprimir los valores IDF
print("Frecuencia Inversa de Documento (IDF):")
print(idf_df)

Frecuencia Inversa de Documento (IDF):
         Término       IDF
0        acceder  2.079442
1     accesibles  2.079442
2         acceso  2.079442
3         ademas  2.079442
4       adopcion  2.079442
..           ...       ...
223        viaje  2.079442
224         vida  1.386294
225  videojuegos  2.079442
226    virtuales  2.079442
227     visuales  2.079442

[228 rows x 2 columns]


In [40]:
# Ordenar los valores de IDF de mayor a menor
idf_ordenado = idf_df.sort_values(by='IDF', ascending=False)

# Imprimir los valores IDF ordenados
print("Frecuencia Inversa de Documento (IDF) ordenada de mayor a menor:")
print(idf_ordenado)

Frecuencia Inversa de Documento (IDF) ordenada de mayor a menor:
       Término       IDF
0      acceder  2.079442
134   medicina  2.079442
136  mejorando  2.079442
138    mejores  2.079442
139    mercado  2.079442
..         ...       ...
73          el  0.133531
126        las  0.133531
53          de  0.000000
66   documento  0.000000
125         la  0.000000

[228 rows x 2 columns]


In [41]:
# Convertir la matriz TF a un arreglo de NumPy para la multiplicación
tf_array = tf_df.values

# Calcular la matriz TF-IDF multiplicando TF por IDF
tfidf_matriz = tf_array * idf_values.values

# Convertir la matriz TF-IDF a un DataFrame para facilitar la visualización
tfidf_df = pd.DataFrame(tfidf_matriz, columns=idf_df['Término'])

# Imprimir la matriz TF-IDF
print("Matriz TF-IDF:")
print(tfidf_df)

Matriz TF-IDF:
Término   acceder  accesibles    acceso    ademas  adopcion  adoptando  \
0        0.000000    0.000000  0.000000  0.000000  0.000000   2.079442   
1        0.000000    0.000000  0.000000  0.000000  0.000000   0.000000   
2        0.000000    0.000000  0.000000  0.000000  0.000000   0.000000   
3        2.079442    0.000000  0.000000  0.000000  0.000000   0.000000   
4        0.000000    0.000000  4.158883  2.079442  0.000000   0.000000   
5        0.000000    0.000000  0.000000  0.000000  0.000000   0.000000   
6        0.000000    0.000000  0.000000  0.000000  2.079442   0.000000   
7        0.000000    2.079442  0.000000  0.000000  0.000000   0.000000   

Término     ahora  alcanzado  algoritmos      anos  ...       una       uno  \
0        0.000000   0.000000    2.079442  0.000000  ...  0.000000  0.000000   
1        2.079442   2.079442    0.000000  0.000000  ...  0.000000  0.000000   
2        0.000000   0.000000    0.000000  0.000000  ...  0.000000  0.000000   
3 

**Procedemos a eliminar las stopwords.**

In [42]:
stopwords = [
    'la', 'las', 'el', 'los', 'de', 'y', 'en', 'a', 'que', 'con', 'por', 'para', 'como', 'más', 'es', 'se', 'esto', 'este', 
    'esta', 'de', 'un', 'una', 'lo', 'al', 'del', 'su', 'las', 'ser', 'está', 'están', 'o', 'pero', 'no', 'si', 'con', 'su', 
    'sobre', 'entre', 'este', 'también', 'esos', 'esas', 'esto', 'ese', 'por', 'ademas', 'cuando', 'ya', 'nos', 'tiene', 'nosotros', 
    'mi', 'tú', 'te', 'su', 'y', 'se', 'un', 'al', 'desde', 'hasta', 'durante', 'aunque', 'al', 'muy', 'hay', 'donde', 'esto', 
    'ser', 'una', 'los', 'mientras', 'cual', 'quien', 'que', 'sido', 'su', 'sobre', 'esto', 'ella', 'esto', 'yo', 'debería', 
    'su', 'mi', 'tiempo', 'tal', 'vez', 'tan', 'uno', 'los', 'hace', 'muchos', 'otros', 'estas', 'ellas', 'nosotros', 'ellas', 
    'algunas', 'algún', 'sólo', 'vez', 'que', 'más', 'del', 'menos', 'todo', 'al', 'siempre', 'por', 'también', 'en', 'todos'
]

In [43]:
# Filtrar las columnas de la matriz TF-IDF para eliminar las stopwords en español
columns_to_keep = [term for term in tfidf_df.columns if term not in stopwords]

# Crear una nueva matriz TF-IDF sin las stopwords
tfidf_filtered_df = tfidf_df[columns_to_keep]

# Imprimir la nueva matriz TF-IDF sin stopwords
print("Matriz TF-IDF sin Stopwords:")
print(tfidf_filtered_df)

Matriz TF-IDF sin Stopwords:
Término   acceder  accesibles    acceso  adopcion  adoptando     ahora  \
0        0.000000    0.000000  0.000000  0.000000   2.079442  0.000000   
1        0.000000    0.000000  0.000000  0.000000   0.000000  2.079442   
2        0.000000    0.000000  0.000000  0.000000   0.000000  0.000000   
3        2.079442    0.000000  0.000000  0.000000   0.000000  0.000000   
4        0.000000    0.000000  4.158883  0.000000   0.000000  0.000000   
5        0.000000    0.000000  0.000000  0.000000   0.000000  0.000000   
6        0.000000    0.000000  0.000000  2.079442   0.000000  0.000000   
7        0.000000    2.079442  0.000000  0.000000   0.000000  0.000000   

Término  alcanzado  algoritmos      anos  aplicaciones  ...  tratamientos  \
0         0.000000    2.079442  0.000000      0.000000  ...      0.000000   
1         2.079442    0.000000  0.000000      0.000000  ...      0.000000   
2         0.000000    0.000000  0.000000      0.000000  ...      0.000000

**3. Procesamiento de las consultas:**

**Preprocesamiento de las consultas de manera similar a los documentos.**

In [44]:
queries = [
    "inteligencia artificial en medicina",
    "beneficios de la educación a distancia",
    "realidad aumentada en videojuegos",
    "desarrollo personal y hábitos saludables",
    "futuro del comercio electrónico",
    "tecnologías en cine moderno",
    "competencias de e-sports",
    "diagnóstico con dispositivos portátiles",
    "literatura de ciencia ficción",
    "plataformas de streaming"
]

# Aplicar la normalización a cada consulta
queries_normalizadas = [normalizar(query) for query in queries]

# Imprimir las consultas normalizadas
print("Consultas normalizadas:")
for idx, query in enumerate(queries_normalizadas):
    print(f"Consulta {idx + 1}: {query}")

Consultas normalizadas:
Consulta 1: inteligencia artificial en medicina
Consulta 2: beneficios de la educacion a distancia
Consulta 3: realidad aumentada en videojuegos
Consulta 4: desarrollo personal y habitos saludables
Consulta 5: futuro del comercio electronico
Consulta 6: tecnologias en cine moderno
Consulta 7: competencias de esports
Consulta 8: diagnostico con dispositivos portatiles
Consulta 9: literatura de ciencia ficcion
Consulta 10: plataformas de streaming


**Representación de las consultas en el espacio vectorial TF-IDF.**

In [45]:
# Aplicar la normalización y tokenización a las consultas
queries_normalizadas = [normalizar(query) for query in queries]

# Tokenizar las consultas normalizadas
queries_tokenizadas = [tokenizar(query) for query in queries_normalizadas]

# Eliminar las stopwords de las consultas
queries_sin_stopwords = []
for query in queries_tokenizadas:
    query_sin_stopwords = [word for word in query if word not in stopwords]
    queries_sin_stopwords.append(query_sin_stopwords)

# Reconvertir las consultas sin stopwords a texto (para pasarlas al vectorizador)
queries_limpias = [' '.join(query) for query in queries_sin_stopwords]

# Inicializar el vectorizador TF-IDF
vectorizer_tfidf = TfidfVectorizer()

# Convertir la matriz a un DataFrame para facilitar la visualización
queries_tfidf_matrix = vectorizer_tfidf.fit_transform(queries_limpias)
queries_tfidf_df = pd.DataFrame(queries_tfidf_matrix.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

# Imprimir la matriz TF-IDF de las consultas
print("Matriz TF-IDF para las consultas:")
print(queries_tfidf_df)

Matriz TF-IDF para las consultas:
   artificial  aumentada  beneficios  ciencia     cine  comercio  \
0     0.57735    0.00000     0.00000  0.00000  0.00000   0.00000   
1     0.00000    0.00000     0.57735  0.00000  0.00000   0.00000   
2     0.00000    0.57735     0.00000  0.00000  0.00000   0.00000   
3     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   
4     0.00000    0.00000     0.00000  0.00000  0.00000   0.57735   
5     0.00000    0.00000     0.00000  0.00000  0.57735   0.00000   
6     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   
7     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   
8     0.00000    0.00000     0.00000  0.57735  0.00000   0.00000   
9     0.00000    0.00000     0.00000  0.00000  0.00000   0.00000   

   competencias  desarrollo  diagnostico  dispositivos  ...  medicina  \
0      0.000000         0.0      0.00000       0.00000  ...   0.57735   
1      0.000000         0.0      0.00000       0.00000  ...   0.00000  

**4. Cálculo de similitudes:**

Cálculo de la similitud entre cada consulta y los documentos del corpus utilizando la similitud del coseno

In [48]:
# Inicializar el vectorizador TF-IDF para los documentos
vectorizer_tfidf = TfidfVectorizer(stop_words=stopwords)  # Usamos el mismo vectorizador para consultas y documentos

# Se convierte el corpus de documentos en su representación TF-IDF
tfidf_matrix = vectorizer_tfidf.fit_transform(corpus)

# Transformar las consultas con el mismo vectorizador
queries_tfidf_matrix = vectorizer_tfidf.transform(queries_limpias)

# Calcular la similitud del coseno entre las consultas y los documentos
similitudes_consultas_documentos = cosine_similarity(queries_tfidf_matrix, tfidf_matrix)

# Convertir la matriz de similitudes a un DataFrame para mejor visualización
similitudes_df = pd.DataFrame(similitudes_consultas_documentos,
                              columns=[f'Documento {i+1}' for i in range(len(corpus))],
                              index=[f'Consulta {i+1}' for i in range(len(queries))])

# Imprimir la matriz de similitudes entre consultas y documentos
print("Matriz de Similitud del Coseno entre consultas y documentos:")
print(similitudes_df)

Matriz de Similitud del Coseno entre consultas y documentos:
             Documento 1  Documento 2  Documento 3  Documento 4  Documento 5  \
Consulta 1      0.131945     0.146161     0.000000     0.000000     0.000000   
Consulta 2      0.000000     0.000000     0.000000     0.272234     0.000000   
Consulta 3      0.000000     0.328565     0.000000     0.000000     0.073791   
Consulta 4      0.000000     0.085397     0.000000     0.000000     0.000000   
Consulta 5      0.000000     0.000000     0.148558     0.000000     0.000000   
Consulta 6      0.000000     0.000000     0.000000     0.000000     0.172705   
Consulta 7      0.000000     0.199871     0.000000     0.000000     0.000000   
Consulta 8      0.000000     0.000000     0.000000     0.000000     0.000000   
Consulta 9      0.000000     0.000000     0.000000     0.000000     0.000000   
Consulta 10     0.000000     0.000000     0.104344     0.000000     0.225337   

             Documento 6  Documento 7  Documento 8  
Consu

**5. Ranking de documentos:**

Ordenar los documentos de mayor a menor similitud para cada consulta y mostrar los documentos más relevantes.

In [79]:
for consulta_idx, consulta in similitudes_df.iterrows():
    consulta_idx_int = int(re.search(r'\d+', consulta_idx).group())  # Extrae el número de la consulta
    
    consulta_texto = queries[consulta_idx_int - 1]  # Obtener el texto original de la consulta
    
    # Imprimir el ranking de documentos para cada consulta
    print(f"\nRanking de documentos para la Consulta {consulta_idx_int} - ({consulta_texto}):")
    
    ranking_documentos = consulta.sort_values(ascending=False)  # Ordenamos de mayor a menor
    
    # Mostrar los documentos más relevantes
    top_documentos = ranking_documentos.head(3)  # Los 3 más relevantes
    for doc_idx, score in top_documentos.items():
        # Mostrar el documento y su similitud con el formato deseado
        print(f"**//** {doc_idx}   Similitud Coseno = {score:.6f} **//**")
        
        # Mostrar el texto del documento correspondiente
        doc_texto = corpus[int(doc_idx.split()[-1]) - 1].strip()  # Obtener el texto del documento
        print(f"{doc_texto}")


Ranking de documentos para la Consulta 1 - (inteligencia artificial en medicina):
**//** Documento 8   Similitud Coseno = 0.278563 **//**
Documento 8: Los avances en la medicina están cambiando la forma en que se diagnostican y tratan las enfermedades. Desde la telemedicina hasta los dispositivos portátiles que monitorean la salud en tiempo real, la tecnología está permitiendo a los médicos ofrecer cuidados más personalizados. El futuro de la medicina es prometedor, con tratamientos más accesibles y eficaces.
**//** Documento 6   Similitud Coseno = 0.159074 **//**
Documento 6: La ciencia ficción es uno de los géneros literarios más populares, explorando temas como el viaje en el tiempo, la inteligencia artificial y la vida en otros planetas. Este género no solo entretiene, sino que también invita a reflexionar sobre el futuro de la humanidad y las implicaciones de los avances tecnológicos.
**//** Documento 2   Similitud Coseno = 0.146161 **//**
Documento 2: El desarrollo de videojuego